<a href="https://colab.research.google.com/github/carlosks/carlosks/blob/main/prepared_data_DESAFIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Conexão com o Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import json

def process_news_summaries_file(file_path, processed_data):
    #Lê um arquivo JSON do nosso dataset, processa cada notícia para formatar conforme o solicitado e adiciona à lista processed_data
    with open(file_path, 'r', encoding='utf-8') as file:
        # Iterate over each line in the file
        for line in file:
            # Try to load each line as a separate JSON object
            try:
                json_data = json.loads(line)

                # Check if the required keys exists in the JSON object
                if "title" in json_data and "content" in json_data:
                    title = json_data["title"]
                    content = json_data["content"]
                    formatted_text = f"SUMMARIZE THIS CONTENT.\n[|title|] {title}[|etitle|]\n\n[|content|]{content}[|econtent|]"
                    processed_data.append({"input": formatted_text})
            # Handle JSONDecodeError if a line is not valid JSON
            except json.JSONDecodeError:
                print(f"Skipping invalid JSON data in line: {line}")



In [5]:

# Lista para armazenar todos os dados processados
processed_data = []

# Adicionar dados processados do arquivo JSON regular à mesma lista
process_news_summaries_file(r'/content/drive/MyDrive/Desafio_Fase_3/arquivos 40 Mega/TRN_Dataset_Part_1.json', processed_data)

# Salvar todos os dados processados em um arquivo JSON
output_filename = r'/content/drive/MyDrive/Desafio_Fase_3/arquivos 40 Mega/titles_dataset_chat_data.json'
with open(output_filename, 'w', encoding='utf-8') as file:
    json.dump(processed_data, file, ensure_ascii=False, indent=4)

print(f"Todos os dados reformatados foram salvos em '{output_filename}'.")

Todos os dados reformatados foram salvos em '/content/drive/MyDrive/Desafio_Fase_3/arquivos 40 Mega/titles_dataset_chat_data.json'.


In [9]:
from typing_extensions import Concatenate
import json



def format_dataset_into_model_input(data):

    def separate_text(full_text):

        # Encontrar os delimitadores

        title_start = full_text.find("[|title|]") + len("[|title|]")

        title_end = full_text.find("[|etitle|]")

        content_start = full_text.find("[|content|]") + len("[|content|]")

        content_end = full_text.find("[|econtent|]")



        # Verificar se todos os delimitadores foram encontrados

        if title_start == -1 or title_end == -1 or content_start == -1 or content_end == -1:

            print(f"Delimitador faltando no texto: {full_text[:50]}...")

            return None, None, None



        # Extrair a instrução, texto de entrada e resposta

        instruction = full_text.split('\n')[0]  # Primeira linha é a instrução

        input_text = full_text[title_start:title_end].strip()  # Extrair texto entre [|News|] e [|eNews|]

        response = full_text[content_start:content_end].strip()  # Extrair resumo entre [|summary|] e [|esummary|]



        return instruction, input_text, response



    # Inicializando as listas para armazenar os dados

    instructions = []

    inputs = []

    outputs = []



    # Processando o dataset

    try:

        for item in data:

            # Acessar diretamente o campo 'input' e extrair o conteúdo

            instruction, input_text, response = separate_text(item['input'])

            if instruction and input_text and response:  # Verificar se todos os textos são válidos

                instructions.append(instruction)

                inputs.append(input_text)

                outputs.append(response)

    except KeyboardInterrupt:

        pass  # Ignorar interrupções de teclado



    # Criar o dicionário final

    formatted_data = {

        "instruction": instructions,

        "input": inputs,

        "output": outputs

    }



    # Salvar o resultado em um arquivo JSON

    with open(OUTPUT_PATH_DATASET, 'w') as output_file:

        json.dump(formatted_data, output_file, indent=4)

In [10]:
DATA_PATH = "/content/drive/MyDrive/Desafio_Fase_3/arquivos 40 Mega/titles_dataset_chat_data.json"
OUTPUT_PATH_DATASET = "/content/drive/MyDrive/Desafio_Fase_3/arquivos 40 Mega/formated_titles_dataset_chat_data.json"

# Load the dataset
with open(DATA_PATH, 'r') as f:
    data = json.load(f)

format_dataset_into_model_input(data)